In [1]:
import pandas as pd
import numpy as np
import lightgbm as lgbm
from sklearn.impute import KNNImputer
from sklearn.preprocessing import StandardScaler
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from scipy.stats import uniform, randint
import os
import warnings
warnings.filterwarnings('ignore')

### Other

In [2]:
def validate_imputation(df, col_name):
    """
    Memvalidasi hasil imputasi dengan memeriksa distribusi dan outlier
    
    Args:
        df: DataFrame dengan data yang telah diimputasi
        col_name: Nama kolom yang divalidasi
    """
    if col_name not in df.columns:
        print(f"  Kolom {col_name} tidak ditemukan dalam data")
        return
    
    # Pisahkan data asli dan diimputasi
    original_mask = df[col_name].notna()
    original_data = df.loc[original_mask, col_name]
    
    # Jika semua data asli (tidak ada yang diimputasi), tidak perlu validasi
    if len(original_data) == len(df):
        print(f"  Kolom {col_name} tidak memiliki nilai yang diimputasi")
        return
    
    # Statistik deskriptif untuk data asli
    orig_mean = original_data.mean()
    orig_std = original_data.std()
    orig_min = original_data.min()
    orig_max = original_data.max()
    orig_median = original_data.median()
    orig_q1 = original_data.quantile(0.25)
    orig_q3 = original_data.quantile(0.75)
    
    # Statistik deskriptif untuk data keseluruhan (termasuk yang diimputasi)
    all_mean = df[col_name].mean()
    all_std = df[col_name].std()
    all_min = df[col_name].min()
    all_max = df[col_name].max()
    all_median = df[col_name].median()
    
    # Cetak statistik perbandingan
    print(f"  Validasi untuk {col_name}:")
    print(f"    Data asli: mean={orig_mean:.4f}, std={orig_std:.4f}, min={orig_min:.4f}, max={orig_max:.4f}, median={orig_median:.4f}")
    print(f"    Data dengan imputasi: mean={all_mean:.4f}, std={all_std:.4f}, min={all_min:.4f}, max={all_max:.4f}, median={all_median:.4f}")
    
    # Periksa apakah ada outlier ekstrem dalam data yang diimputasi
    # Definisikan outlier sebagai nilai di luar 3x IQR dari data asli
    iqr = orig_q3 - orig_q1
    lower_bound = orig_q1 - 3 * iqr
    upper_bound = orig_q3 + 3 * iqr
    
    # Data hasil imputasi
    imputed_mask = ~original_mask
    imputed_data = df.loc[imputed_mask, col_name]
    
    # Periksa jumlah outlier
    n_outliers = ((imputed_data < lower_bound) | (imputed_data > upper_bound)).sum()
    
    if n_outliers > 0:
        pct_outliers = (n_outliers / len(imputed_data)) * 100
        print(f"    Peringatan: {n_outliers} nilai hasil imputasi ({pct_outliers:.1f}%) berada di luar batas normal")
        
        # Jika ada outlier ekstrem, lakukan clipping
        if pct_outliers > 5:
            print(f"    Melakukan clipping pada outlier ekstrem...")
            df.loc[imputed_mask & (df[col_name] < lower_bound), col_name] = lower_bound
            df.loc[imputed_mask & (df[col_name] > upper_bound), col_name] = upper_bound
            print(f"    Setelah clipping: min={df.loc[imputed_mask, col_name].min():.4f}, max={df.loc[imputed_mask, col_name].max():.4f}")
    else:
        print("    Tidak ditemukan outlier dalam hasil imputasi")
    
    # Periksa apakah distribusi data telah berubah secara signifikan
    mean_diff_pct = abs((all_mean - orig_mean) / orig_mean) * 100
    std_diff_pct = abs((all_std - orig_std) / orig_std) * 100
    
    if mean_diff_pct > 10 or std_diff_pct > 20:
        print(f"    Peringatan: Distribusi data mungkin telah berubah (mean diff={mean_diff_pct:.1f}%, std diff={std_diff_pct:.1f}%)")
    else:
        print(f"    Distribusi data tetap konsisten setelah imputasi")
        
    return df


# Asumsikan data sudah dimuat dalam DataFrame
# Jika belum, gunakan code berikut untuk memuat data
df = pd.read_csv('data/fitur_clean/anomaly.csv')

def load_data(file_path):
    """Memuat data dari file CSV"""
    df = pd.read_csv(file_path)
    return df

def check_missing_values(df):
    """Menampilkan jumlah nilai yang hilang untuk setiap kolom"""
    missing_values = df.isnull().sum()
    return missing_values[missing_values > 0]

def impute_total_alkalinity(df):
    """
    Mengimputasi total_alkalinity berdasarkan salinitas dan suhu air
    menggunakan rumus empiris
    """
    # Membuat mask untuk mengidentifikasi nilai yang akan diimputasi
    mask = df['total_alkalinity (µmol kg-1)'].isna()
    
    # Mengimputasi berdasarkan formula empiris
    df.loc[mask, 'total_alkalinity (µmol kg-1)'] = (
        2305 + 
        58.66 * (df.loc[mask, 'salinity_50m'] - 35) + 
        2.32 * (df.loc[mask, 'salinity_50m'] - 35)**2 - 
        1.41 * (df.loc[mask, 'water_temperature_50m'] - 20) + 
        0.04 * (df.loc[mask, 'water_temperature_50m'] - 20)**2
    )
    
    return df

def impute_bottom_current_shear_stress(df):
    """
    Mengimputasi bottom_current_shear_stress berdasarkan
    densitas air, koefisien drag, dan kecepatan arus dalam
    """
    mask = df['bottom_current_shear_stress (Pa)'].isna()
    
    # Koefisien drag tipikal untuk dasar laut
    Cd = 0.0025
    
    # Mengimputasi menggunakan rumus τ = ρ * Cd * U^2
    df.loc[mask, 'bottom_current_shear_stress (Pa)'] = (
        df.loc[mask, 'perceived_water_density'] * 
        Cd * 
        df.loc[mask, 'current_velocity_deep']**2
    )
    
    return df

def impute_sound_speed_water(df):
    """
    Mengimputasi kecepatan suara dalam air menggunakan persamaan UNESCO
    """
    mask = df['sound_speed_water (m s-1)'].isna()
    
    # Menggunakan kedalaman untuk perhitungan (estimasi dari hydrostatic_pressure)
    # Jika hydrostatic_pressure tidak tersedia, gunakan nilai default untuk D
    if 'hydrostatic_pressure' in df.columns and not df['hydrostatic_pressure'].isna().all():
        # Estimasi kedalaman dari tekanan hidrostatik (approx: 1 dbar ≈ 1 meter)
        D = df.loc[mask, 'hydrostatic_pressure'] / 10000  # konversi ke dbar
    else:
        # Nilai default kedalaman jika tidak ada data tekanan
        D = 50  # asumsi kedalaman 50m berdasarkan beberapa kolom yang diukur pada 50m
    
    # Imputasi menggunakan rumus UNESCO
    T = df.loc[mask, 'water_temperature_50m']
    S = df.loc[mask, 'salinity_50m']
    
    df.loc[mask, 'sound_speed_water (m s-1)'] = (
        1449.2 + 4.6*T - 0.055*T**2 + 0.00029*T**3 + 
        (1.34 - 0.01*T)*(S - 35) + 0.016*D
    )
    
    return df

def impute_brunt_vaisala(df):
    """
    Mengimputasi Brunt-Väisälä frequency squared berdasarkan
    densitas air dan mixed layer depth
    """
    mask = df['Brunt_Vaisala_frequency_squared (s-2)'].isna()
    
    # Gravitasi
    g = 9.81  # m/s^2
    
    # Estimasi gradien densitas vertikal
    # Karena kita hanya memiliki satu pengukuran densitas, kita perlu membuat estimasi
    # Kita dapat menggunakan mixed_layer_depth dan asumsi gradien densitas
    
    # Perbedaan densitas tipikal antara permukaan dan bawah mixed layer
    delta_rho = 0.5  # kg/m^3, nilai tipikal untuk stratifikasi laut
    
    # Estimasi N^2 = (g/ρ) * (dρ/dz)
    df.loc[mask, 'Brunt_Vaisala_frequency_squared (s-2)'] = (
        g / df.loc[mask, 'perceived_water_density'] * 
        (delta_rho / df.loc[mask, 'mixed_layer_depth (m)'])
    )
    
    return df

def impute_with_lgbm(df, target_column, n_iterations=20, cv_folds=3):
    """
    Mengimputasi nilai menggunakan LightGBM dengan hyperparameter tuning melalui Random Search
    
    Args:
        df: DataFrame dengan data
        target_column: Nama kolom yang akan diimputasi
        n_iterations: Jumlah iterasi untuk Random Search
        cv_folds: Jumlah lipatan cross-validation
        
    Returns:
        DataFrame dengan nilai yang telah diimputasi
    """
    # Membuat salinan data
    df_temp = df.copy()
    
    # Identifikasi nilai yang hilang
    mask = df_temp[target_column].isna()
    
    # Jika tidak ada nilai yang hilang, kembalikan dataframe asli
    if not mask.any():
        return df
    
    # Pilih fitur untuk model
    # Hindari kolom dengan nilai yang hilang dan kolom yang tidak relevan
    exclude_cols = ['measurement_id', 'depth_reading_time', 'hydrostatic_pressure', 
                    'seafloor_pressure', 'year', 'month', 'day', 'hour', 'dayofweek',
                    target_column]
    
    # Tambahkan kolom dengan missing values tinggi ke exclude_cols
    for col in df.columns:
        if col not in exclude_cols and df[col].isna().sum() > 0.3 * len(df):
            exclude_cols.append(col)
    
    feature_cols = [col for col in df_temp.columns if col not in exclude_cols]
    
    # Bagi data menjadi set pelatihan dan set target
    df_train = df_temp[~mask]
    X_train = df_train[feature_cols]
    y_train = df_train[target_column]
    
    # Definisi rentang hyperparameter untuk random search
    param_distributions = {
        'num_leaves': randint(20, 200),
        'learning_rate': uniform(0.01, 0.29),
        'n_estimators': randint(100, 500),
        'max_depth': randint(3, 12),
        'min_child_samples': randint(5, 100),
        'subsample': uniform(0.6, 0.4),
        'colsample_bytree': uniform(0.6, 0.4),
        'reg_alpha': uniform(0, 3),
        'reg_lambda': uniform(0, 3),
    }
    
    # Buat model LightGBM dasar
    base_model = lgbm.LGBMRegressor(
        objective='regression',
        n_jobs=-1,
        random_state=42,
        verbose=-1,
        importance_type='gain'
    )
    
    # Lakukan random search untuk hyperparameter tuning
    print(f"Melakukan hyperparameter tuning untuk {target_column} dengan Random Search...")
    rs = RandomizedSearchCV(
        estimator=base_model,
        param_distributions=param_distributions,
        n_iter=n_iterations,
        cv=cv_folds,
        scoring='neg_mean_squared_error',
        verbose=0,
        random_state=42,
        n_jobs=-1
    )
    
    # Train model dengan hyperparameter terbaik
    rs.fit(X_train, y_train)
    
    # Tampilkan parameter terbaik
    best_params = rs.best_params_
    print(f"Parameter terbaik: {best_params}")
    
    # Buat model dengan parameter terbaik
    best_model = lgbm.LGBMRegressor(**best_params, random_state=42)
    best_model.fit(X_train, y_train)
    
    # Fitur importance
    feature_importance = pd.DataFrame({
        'Feature': X_train.columns,
        'Importance': best_model.feature_importances_
    }).sort_values('Importance', ascending=False)
    
    print(f"Top 5 fitur paling penting untuk {target_column}:")
    print(feature_importance.head(5))
    
    # Prediksi nilai yang hilang
    X_missing = df_temp.loc[mask, feature_cols]
    df.loc[mask, target_column] = best_model.predict(X_missing)
    
    return df

def impute_with_knn(df, target_column, n_neighbors=5):
    """
    Mengimputasi nilai menggunakan KNN Imputer
    """
    # Pilih fitur untuk model (hindari kolom dengan missing values dan kolom non-numerik)
    exclude_cols = ['measurement_id', 'depth_reading_time', 'hydrostatic_pressure', 
                  'seafloor_pressure', 'year', 'month', 'day', 'hour', 'dayofweek']
    
    feature_cols = [col for col in df.columns if col not in exclude_cols and 
                   df[col].dtype in ['float64', 'int64'] and 
                   col != target_column]
    
    # Skalakan fitur
    scaler = StandardScaler()
    df_scaled = pd.DataFrame(
        scaler.fit_transform(df[feature_cols]),
        columns=feature_cols
    )
    
    # Tambahkan kolom target
    df_scaled[target_column] = df[target_column]
    
    # Imputasi KNN
    imputer = KNNImputer(n_neighbors=n_neighbors)
    df_imputed = pd.DataFrame(
        imputer.fit_transform(df_scaled),
        columns=df_scaled.columns
    )
    
    # Kembalikan nilai yang diimputasi ke dataframe asli
    df[target_column] = df_imputed[target_column]
    
    return df

def impute_with_mice(df, target_columns):
    """
    Mengimputasi nilai menggunakan Multiple Imputation by Chained Equations (MICE)
    """
    # Pilih kolom untuk imputasi
    feature_cols = [col for col in df.columns if df[col].dtype in ['float64', 'int64'] and
                   col not in ['measurement_id', 'depth_reading_time', 'hydrostatic_pressure', 'seafloor_pressure']]
    
    # Inisialisasi MICE
    mice_imputer = IterativeImputer(max_iter=15, random_state=42)
    
    # Lakukan imputasi
    df_imputed = df.copy()
    df_imputed[feature_cols] = mice_imputer.fit_transform(df[feature_cols])
    
    # Kembalikan nilai yang diimputasi untuk kolom target ke dataframe asli
    for col in target_columns:
        if col in feature_cols:
            df[col] = df_imputed[col]
    
    return df

def evaluate_imputation(df, target_column, imputation_method, test_size=0.2):
    """
    Mengevaluasi metode imputasi dengan menyembunyikan sebagian data yang ada 
    dan membandingkan nilai prediksi dengan nilai sebenarnya
    """
    # Salin data yang memiliki nilai (tidak NA) di kolom target
    df_complete = df[df[target_column].notna()].copy()
    
    # Bagi data menjadi training dan testing
    train_data, test_data = train_test_split(df_complete, test_size=test_size, random_state=42)
    
    # Salin nilai asli dari data pengujian
    original_values = test_data[target_column].copy()
    
    # Set nilai kolom target pada data pengujian menjadi NA
    test_data[target_column] = np.nan
    
    # Gabungkan data pelatihan dan pengujian
    combined_data = pd.concat([train_data, test_data])
    
    # Lakukan imputasi
    if imputation_method == 'lgbm':
        imputed_data = impute_with_lgbm(combined_data, target_column, n_iterations=10, cv_folds=3)
    elif imputation_method == 'knn':
        imputed_data = impute_with_knn(combined_data, target_column)
    elif imputation_method == 'mice':
        imputed_data = impute_with_mice(combined_data, [target_column])
    else:
        raise ValueError("Metode imputasi tidak valid")
    
    # Ambil nilai yang diimputasi
    imputed_values = imputed_data.loc[test_data.index, target_column]
    
    # Hitung metrik evaluasi
    mse = np.mean((original_values - imputed_values) ** 2)
    rmse = np.sqrt(mse)
    mae = np.mean(np.abs(original_values - imputed_values))
    r2 = 1 - (np.sum((original_values - imputed_values) ** 2) / 
              np.sum((original_values - np.mean(original_values)) ** 2))
    
    return {
        'MSE': mse,
        'RMSE': rmse,
        'MAE': mae,
        'R2': r2
    }

def main(file_path):
    """
    Fungsi utama untuk memproses dan mengimputasi data
    """
    # Muat data
    df = load_data(file_path)
    
    # Periksa nilai yang hilang sebelum imputasi
    print("Nilai yang hilang sebelum imputasi:")
    print(check_missing_values(df))
    
    # Daftar kolom yang perlu diimputasi
    columns_to_impute = [
        'total_alkalinity (µmol kg-1)',
        'bottom_current_shear_stress (Pa)',
        'sound_speed_water (m s-1)',
        'Brunt_Vaisala_frequency_squared (s-2)'
    ]
    
    # Evaluasi dan pilih metode terbaik untuk setiap kolom
    results = {}
    methods = ['lgbm', 'knn', 'mice']
    
    for col in columns_to_impute:
        # Jika kolom memiliki cukup data untuk dievaluasi
        if df[col].notna().sum() > 100:  # Minimal 100 data untuk evaluasi
            print(f"\nEvaluasi metode untuk {col}:")
            col_results = {}
            
            for method in methods:
                try:
                    result = evaluate_imputation(df, col, method)
                    col_results[method] = result
                    print(f"  {method}: RMSE = {result['RMSE']:.4f}, MAE = {result['MAE']:.4f}, R² = {result['R2']:.4f}")
                except Exception as e:
                    print(f"  {method}: Error - {str(e)}")
            
            # Pilih metode dengan RMSE terendah
            best_method = min(col_results, key=lambda x: col_results[x]['RMSE'])
            results[col] = {'best_method': best_method, 'metrics': col_results[best_method]}
            print(f"  Metode terbaik: {best_method} dengan RMSE = {col_results[best_method]['RMSE']:.4f}")
        else:
            # Jika data tidak cukup untuk evaluasi, gunakan pendekatan berbasis fisika
            results[col] = {'best_method': 'physics_based', 'metrics': None}
            print(f"\n{col}: Menggunakan pendekatan berbasis fisika karena data tidak cukup untuk evaluasi")
    
    # Lakukan imputasi menggunakan metode terbaik atau metode berbasis fisika
    for col in columns_to_impute:
        print(f"\nMengimputasi {col} menggunakan {results[col]['best_method']}...")
        
        if results[col]['best_method'] == 'physics_based':
            if col == 'total_alkalinity (µmol kg-1)':
                df = impute_total_alkalinity(df)
            elif col == 'bottom_current_shear_stress (Pa)':
                df = impute_bottom_current_shear_stress(df)
            elif col == 'sound_speed_water (m s-1)':
                df = impute_sound_speed_water(df)
            elif col == 'Brunt_Vaisala_frequency_squared (s-2)':
                df = impute_brunt_vaisala(df)
        elif results[col]['best_method'] == 'lgbm':
            # Untuk LGBM final, gunakan lebih banyak iterasi untuk random search
            df = impute_with_lgbm(df, col, n_iterations=90, cv_folds=5)
        elif results[col]['best_method'] == 'knn':
            df = impute_with_knn(df, col)
        elif results[col]['best_method'] == 'mice':
            df = impute_with_mice(df, [col])
    
    # Periksa nilai yang hilang setelah imputasi
    print("\nNilai yang hilang setelah imputasi:")
    print(check_missing_values(df))
    
    # Validasi hasil imputasi
    print("\nValidasi hasil imputasi:")
    for col in columns_to_impute:
        validate_imputation(df, col)
    
    # Simpan hasil
    output_path = "data/fitur_clean/other.csv"
    df.to_csv(output_path, index=False)
    print(f"\nData yang telah diimputasi disimpan ke: {output_path}")
    
    return df

# Contoh penggunaan:
if __name__ == "__main__":
    # Ganti dengan path file CSV Anda
    file_path = "data/fitur_clean/anomaly.csv"  
    imputed_df = main(file_path)

Nilai yang hilang sebelum imputasi:
seafloor_pressure                        4433
total_alkalinity (µmol kg-1)             1512
bottom_current_shear_stress (Pa)         1379
sound_speed_water (m s-1)                2528
Brunt_Vaisala_frequency_squared (s-2)    1676
hydrostatic_pressure                     6567
dtype: int64

Evaluasi metode untuk total_alkalinity (µmol kg-1):
Melakukan hyperparameter tuning untuk total_alkalinity (µmol kg-1) dengan Random Search...
Parameter terbaik: {'colsample_bytree': 0.7801997007878172, 'learning_rate': 0.013846838736361293, 'max_depth': 11, 'min_child_samples': 64, 'n_estimators': 113, 'num_leaves': 28, 'reg_alpha': 0.04789875666064258, 'reg_lambda': 0.692681476866447, 'subsample': 0.6964101864104046}
Top 5 fitur paling penting untuk total_alkalinity (µmol kg-1):
                                  Feature  Importance
40              sound_speed_water (m s-1)         178
45  Brunt_Vaisala_frequency_squared (s-2)         130
39       bottom_current_sh